<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Ejercicios creados a partir de 2019-2 por Equipo Docente IIC2233. </font>
<font size='1'> Actualizados en 2020-1.</font>
</p>


# Ejercicios propuestos: Networking

Los siguientes problemas se proveen como oportunidad de ejercitar los conceptos revisados en el material de **Networking**. Si tienes dudas sobre algún problema o alguna solución, no dudes en dejar una *issue* en el [foro del curso](https://github.com/IIC2233/syllabus/issues).

Los ejercicios se subdividen según la numeración de los cuadernos de la semana.

## Conceptos básicos de redes de datos (*networking*)

### Ejercicio 1.1: `Bot Whastapp OMS: funcionalidades básicas`

Si guardas el número +41 79 893 18 92 en tu celular y le envías un mensaje a través de Whatsapp, entrarás en contacto con el bot oficial de la Organización Mundial de la Salud que se encarga informar a la gente con datos actualizados del Cov-19. A través de un menú, el bot te permite seleccionar distinta información. 


En este ejercicio, simularemos el trabajo del bot de la OMS a través de Networking, donde el Bot actúa como el servidor que le provee la información del 9 de junio 2020 a las distintas personas (clientes).


En esta primera parte, el bot deberá proveer la siguiente información a un usuario: 

    [1] Casos en Chile
    [2] Casos en el Mundo
    [3] Consejos contra el Coronavirus
    [4] Reportar un nuevo caso (en 1.2 hablaremos más sobre esta opción)

La idea es que si el usuario selecciona una opción [1, 2, 3, 4] en su consola, el bot le envíe la información asociada asociada a dicha solicitud. Esa información ya se encuentra cargada y la puedes encontrar en los values del diccionario self.info del servidor. Además si el usuario ingresa 0, el bot le debe enviar la visualización del menú (también está contenida en self.info). En caso de que el usuario no envíe una opción [0, 1, 2, 3, 4], se le debe enviar el aviso: 'No seleccionaste ninguna opción. Envía 0 para recibir el menú de opciones.'

Para lograr esto, deberás implementar solamente el método `handle_command(opcion, client_socket)` del servidor. Este recibe la opción 'opcion' que le envía el usuario del socket 'client_socket' y dependiendo del valor de la opción debe enviarle de vuelta la respuesta correspondiente. A continuación se muestra el código para el bot y el usuario.



In [ ]:
import socket
import threading


class Bot:

    def __init__(self, port, host):
        print("Inicializando servidor...")

        self.host = host
        self.port = port
        self.socket_server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.bind_and_listen()
        self.accept_connections()
        
        self.nuevo_caso_chile = 138846
        self.nuevo_caso_mundo = 108918
        self.casos_chile = 7039918
        self.casos_mundo = 4696
        self.actualizar_info()

    def bind_and_listen(self):

        self.socket_server.bind((self.host, self.port))
        self.socket_server.listen()
        print(f"Servidor escuchando en {self.host}:{self.port}...")

    def accept_connections(self):

        thread = threading.Thread(target=self.accept_connections_thread)
        thread.start()

    def accept_connections_thread(self):

        print("Servidor aceptando conexiones...")

        while True:
            client_socket, _ = self.socket_server.accept()
            listening_client_thread = threading.Thread(
                target=self.listen_client_thread,
                args=(client_socket, ),
                daemon=True)
            listening_client_thread.start()
            

    @staticmethod
    def send(value, sock):

        stringified_value = str(value)
        msg_bytes = stringified_value.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        sock.send(msg_length + msg_bytes)
    
    def sendall(self, value):
        pass

    def listen_client_thread(self, client_socket):

        print("Servidor conectado a un nuevo cliente...")
        
        while True:
            response_bytes_length = client_socket.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(client_socket.recv(read_length))

            received = response.decode()
            self.handle_command(received, client_socket)
            

    def handle_command(self, opcion, client_socket):
        pass

    def actualizar_info(self):
        self.info = {'0' : '\n Menú de opciones \n' \
                            '[1] Casos en Chile \n' \
                            '[2] Casos en el Mundo \n' \
                            '[3] Consejos contra el Coronavirus \n' \
                            '[4] Reportar un nuevo caso',
                     '1' : f' Total (new) cases  \n Chile \n {self.casos_chile} ' \
                           f'confirmed cases ({self.nuevo_caso_chile}) \n 2264 deaths (627)',
                     '2' : f' Total (new) cases  \n Global \n {self.casos_mundo} ' \
                           f'confirmed cases ({self.nuevo_caso_mundo}) \n 404396 deaths (3539)',
                     '3' : 'Consejos \n' \
                            ' - Wash your hands frequently \n' \
                            ' - Avoid touching your eyes, mouth and nose \n' \
                            ' - Cover your mouth and nose with your bent elbow or tissue when you cough or sneeze \n' \
                            ' - Avoid crowded places \n' \
                            ' - Stay at home if you feel unwell - even with a slight fever and cough \n' \
                            ' - If you have a fever, cough and difficulty breathing, seek medical care early - but call by phone first',
                     '4' : 'Se reportó un nuevo caso en Chile '
                }
    def actualizar_casos(self):
        pass


if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    server = Bot(port, host)

In [ ]:
import socket
import threading


class Usuario:

    def __init__(self, port, host):

        self.host = host
        self.port = port
        self.socket_client = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

        try:
            self.connect_to_server()
            self.listen()
            self.repl()
        except ConnectionError:
            print("Conexión terminada.")
            self.socket_client.close()
            exit()

    def connect_to_server(self):

        self.socket_client.connect((self.host, self.port))

    def listen(self):


        thread = threading.Thread(target=self.listen_thread, daemon=True)
        thread.start()

    def send(self, msg):

        msg_bytes = msg.encode()
        msg_length = len(msg_bytes).to_bytes(4, byteorder='big')
        self.socket_client.sendall(msg_length + msg_bytes)

    def listen_thread(self):
        while True:
            response_bytes_length = self.socket_client.recv(4)
            response_length = int.from_bytes(
                response_bytes_length, byteorder='big')
            response = bytearray()

            while len(response) < response_length:
                read_length = min(4096, response_length - len(response))
                response.extend(self.socket_client.recv(read_length))

            print(f"{response.decode()}\n>>> ", end='')

    def repl(self):


        print("------ Whatsapp WHO ------\n Apreta 0 para recibir el menú de opciones\n>>> ", end='')
        while True:
            msg = input()
            response = self.send(msg)



if __name__ == "__main__":
    port = 8080
    host = socket.gethostname()

    client = Usuario(port, host)

### Ejercicio 1.2: `Opción actualizar casos`

Ahora que implementaste `handle_command`, notas que cuando reportas un nuevo caso, no se contabiliza en la base de datos ni le avisa a otras personas conectadas. Para ello, vas a tener que modificar `handle_command` e implementar `sendall` y `actualizar_casos`. La idea es que cuando algún usuario reporte un nuevo contagiado en Chile, el método `actualizar_casos` actualice en el servidor la cantidad de contagiados del día y en total en Chile y en el mundo. Después a través del método `sendall` se deberá enviar el aviso a todos los usuarios conectados que hay un nuevo contagiado (piensa en alguna forma de guardar a los sockets cuando se conectan. Para este ejercicio puedes asumir que una vez que un usuario conectado nunca se desconectará). Si algún usuario quiere visualizar la información de los contagiados esta debe estar actualizada correctamente. Reutiliza el código en 1.1.

## Networking con Python

### Ejercicio 2.1: HTTP Requests y Perritos!

En este ejercicio deberás comunicarte con un servidor en internet! Usaremos la API [dog.ceo](https://dog.ceo/dog-api/) para descargar una foto aleatoria de un perrito en nuestro computador! Para conectarse a páginas web, es común que se usen los protocolos `HTTP` y `HTTPS`. En general las páginas web requieren canales seguros para la comunicación, por lo que usaremos `HTTPS`. Tu trabajo será comunicarte con el servidor en la nube usando **sockets** y *HTTP Requests*, y recibir e interpretar las *HTTP Responses* que nos mandará de vuelta, para obtener nuestra preciada foto de un perrito!

Para trabajar usando `HTTP` Requests, se te entregan las siguientes funciones: 
- `extraer_http_headers`: Recibe un **bytestring** que contiene los headers de la *HTTP request/response*, y retorna un diccionario con los pares `header` : `contenidos`. No es necesario que la apliques, esta se usa dentro de las siguientes funciones.
- `generar_http_request`: Recibe tres argumentos: El host al cual contactar, y el path al cual acceder. Por ejemplo, si [https://github.com/IIC2233/syllabus/issues](https://github.com/IIC2233/syllabus/issues) es la URL, el host corresponde a `github.com` y el path corresponde a `IIC2233/syllabus/issues`. Retorna un **string** con la *HTTP Request* formada.
- `recibir_http_response`: Recibe un **socket** como argumento. Recibe una *HTTP Response* desde el host. Esta función retorna un **bytestring**, que contiene el mensaje enviado desde el servidor.

A continuación, se muestran las funciones descritas:

In [1]:
# No debes modificar este código, pero eres bienvenido a revisarlo para ver como funciona
import json
from time import sleep

def generar_http_request(host, path="", method="GET"):
    """
    Retorna una string con formato de HTTP Request, lista para ser enviada.
    Recuerda que las HTTP requests tienen el siguiente formato:
    ```
    GET /path HTTP/1.1
    Host: direccion_host
    ```
    """
    request = f"{method} /{path} HTTP/1.1\r\n"
    request += f"Host: {host}\r\n\r\n"
    return request


def recibir_http_response(sock):
    """
    Retorna una bytestring con la HTTP Response recibida del host.
    Las HTTP response tienen un formato similar:
    ```
    GET /path HTTP/1.1
    Host: direccion_host
    Content-Length: 12356789
    Otro-Header: ...

    Mensaje recibido...
    ```
    Notar que luego de una línea en blanco, viene el mensaje recibido. Este puede ser cualquier
    secuencia de bytes, como texto, un JSON, o una imagen.
    ESTA FUNCIÓN RETORNA DICHO MENSAJE RECIBIDO
    """
    buffer = sock.recv(4096) # Este primer buffer contiene parte headers y parte datos

    headers, datos = buffer.split(b'\r\n\r\n') # una línea en blanco indica su separación
    
    if b'\r\n' in datos:
        datos = datos.split(b'\r\n')[1]
        
    dict_headers = extraer_http_headers(headers)
    
    # El tamaño en bytes del mensaje está en el header Content-Length
    if "Content-Length" in dict_headers:
        tamano_mensaje = int(dict_headers['Content-Length'])
    else:
        tamano_mensaje = len(datos)

    tamano_descargado = len(datos) # Ya descargamos la primera parte (venía junto con los headers)
    print(f"El tamaño del mensaje es de {tamano_mensaje} bytes y tenemos {tamano_descargado} datos")
    # A continuación, se descarga el resto
    if tamano_descargado < tamano_mensaje:
        print("Descargando datos ....")
    while tamano_descargado < tamano_mensaje:
        print("Descargando... {:3d}%".format(round(100*tamano_descargado/tamano_mensaje)), end="\r")
        buffer = sock.recv(2024)
        datos += buffer
        tamano_descargado += len(buffer)
        sleep(0.2)
    print("El mensaje ha sido recibido!")
    return datos


def extraer_http_headers(headers_string):
    """
    Transforma un string que contiene los headers de un http response (headers_string)
    a un diccionario, el cual contiene pares de la forma { nombre_header : datos }
    Retorna el diccionario descrito anteriormente.
    """
    headers_list = headers_string.split(b'\r\n')
    protocol, code, code_name = headers_list[0].split(b' ')
    headers = {'protocol': protocol, 'code': code, 'code_name': code_name}
    for header in headers_list[1:]:
        name, data = header.split(b': ')
        headers[name.decode()] = data.decode()
    return headers

Ahora, debes completar el siguiente código, aplicando la materia vista en los `notebooks`. Es muy recomendable que uses los comentarios para guiarte.

In [ ]:
# Release
import ssl # Permite establecer una conexión segura para el protocolo HTTPS
import socket
import json


if __name__ == "__main__":
    host = "dog.ceo" # Este es el host que usaremos. En general puede ser una IP o una URL
    path = "api/breeds/image/random" # Ruta del host a la cual queremos acceder
    # COMPLETAR (¿Cuál es el puerto que se usa para una conexión HTTPS?)
    port = 

    # Crea un socket en la variable socket_https
    # Necesitamos que la conexión sea usando IPv4, y queremos hacerla a través de TCP
    socket_https = 

    # Estas líneas hacen que la conexión funcione sobre HTTPS, no debes modificarlas
    context = ssl.SSLContext(ssl.PROTOCOL_TLS)
    socket_https = context.wrap_socket(socket_https, server_hostname=host)
    socket_https.settimeout(10) # Obliga al socket a lanzar una excepción al pasar N segundos

    try:
        # Para las siguientes partes, recuera que tienes las funciones 
        # generar_http_request y recibir_http_response entregadas anteriormente
        # COMPLETAR
        # CONECTAR SOCKET AQUÍ (El socket a utilizar sigue en la variable socket_https)
        
        
        # GENERAR Y ENVIAR HTTP REQUEST AQUÍ
        
        
        
        # RECIBIR HTTP RESPONSE AQUÍ
        
        
        # La respuesta debería ser un JSON, por lo que debes decodificarla a continuación
        json_obj = 
        
    except ConnectionError as err:
        print("Ocurrió un error")
    finally:
        # CERRAR EL SOCKET UTILIZADO AQUÍ
        
        

    if json_obj['status'] != "success":
        print("Ha ocurrido un error!")
        exit(1)

    # El JSON recibido contiene una nueva url que usaremos
    url = json_obj['message']
    host, path = url[8:].split("/", 1)
    print(f"Nueva URL: {url}\nNuevo host: {host}\nNuevo path: {path}")

    # Debido a que el host es distinto, debemos abrir un nuevo socket
    # COMPLETAR
    # Crea un socket en la variable socket_https, de manera análoga a la anterior
    
    
    # Nuevamente, estas líneas hacen que la conexión funcione sobre HTTPS, no debes modificarlas
    context = ssl.SSLContext(ssl.PROTOCOL_TLSv1)
    socket_https = context.wrap_socket(socket_https, server_hostname=host)
    socket_https.settimeout(10) # Obliga al socket a lanzar una excepción al pasar N segundos

    try:
        # COMPLETAR
        # CONECTAR SOCKET AQUÍ
        
        
        # GENERAR Y ENVIAR HTTP REQUEST AQUÍ
        
        
        
        # RECIBIR HTTP RESPONSE AQUÍ
        

        # Finalmente, guardamos los datos recibidos en un archivo con formato jpg
        with open('perrito.jpg', 'wb') as image_file:
            image_file.write(response)
            print("La imagen se ha guardado en perrito.jpg !")
    except ConnectionError as err:
        print("Ocurrió un error")
    finally:
        # CERRAR EL SOCKET UTILIZADO AQUÍ
        
        

### Ejercicio 2.2: TCP + UDP = 🕒❤️🌍

En este ejercicio mezclaremos la robustez de TCP con la velocidad de UDP! Deberas implementar un **servidor** y un **cliente**, de tal manera que se comuniquen usando ambos protocolos. 

El **cliente** poseerá una lista de ciudades alrededor del mundo, y eligirá una para obtener su hora en tiempo real. Este le enviará un comando al servidor mediante TCP, e inmediatamente recibirá la hora en tiempo real por medio de UDP!
El **servidor** recibirá los comandos provenientes del **cliente** (mediante TCP), y le enviará la hora correspondiente en tiempo real (por medio de UDP).

Se definen 3 tipos de comando que el **cliente** le puede enviar al **servidor**:
 - `"hora"`: Este comando le pide al servidor que se envíe la hora de una ciudad. Junto con este, se enviará la ciudad de la cual se quiere obtener la hora y el puerto en donde recibirla. El servidor deberá encargarse de enviarla luego de recibir este comando.
 - `"terminar"`: Este comando le pide al servidor que se deje de enviar la hora.
 - `"cerrar"`: Este comando cerrará el servidor.

Los mensajes enviados hacia el **servidor** deben estar en JSON y cumplir el siguiente formato
```python
{
    "comando" : nombre_comando,
    "ciudad"  : nombre_ciudad,
    "puerto"  : puerto_udp
}
```
Donde `nombre_comando` corresponde a uno de los listados arriba. Si el comando es `"hora"`, también se deberá incluir `"ciudad"` y `"puerto"`. `nombre_ciudad` corresponde al nombre de la ciudad seleccionada, y `puerto_udp` corresponde al puerto al cual el **servidor** debe enviar los mensajes por medio de UDP (notar que este lo elige el **cliente** aleatoriamente, y es distinto al `puerto_tcp`).

A continuación, se muestra el código del **cliente** que deberás completar:

In [ ]:
# Release
import sys
import socket
import json
from random import randint

opciones = {
    "1" : "Santiago",
    "2" : "Sao Paulo",
    "3" : "Ciudad de México",
    "4" : "New York",
    "5" : "Los Angeles",
    "6" : "Madrid",
    "7" : "Londres",
    "8" : "Berlin",
    "9" : "Moscú",
    "10" : "Ciudad del Cabo",
    "11" : "Delhi",
    "12" : "Beijing",
    "13" : "Singapur",
    "14" : "Sydney",
    "0" : "Salir"
}

# Estos son el host y el puerto que usará el socket TCP
host_tcp = socket.gethostname()
puerto_tcp = 35020

# A continuación debes crear el socket TCP
# COMPLETAR
socket_tcp = 

try:
    # Acá debes conectar el socket
    # COMPLETAR
    
    
except ConnectionError:
    # Si falla, no te olvides de cerrar el socket aquí!
    # COMPLETAR
    
    
    print(f"No se pudo conectar a {host_tcp}:{puerto_tcp} :(")
    sys.exit(1)

opcion = None
while True:
    for i in opciones:
        print(f"[{i}] {opciones[i]}")
    opcion = input("Ingrese una opción de la lista: ")
    if not opciones.get(opcion):
        print("\nEsta opción no está disponible\n")
        continue
    if opciones[opcion] == "Salir":
        # Si se elige la opción para salir, deberás enviar el comando "cerrar"
        # COMPLETAR
        
        
        
        
        print("\nSaliendo del programa...\n")
        break
    # Primero, se elige al azar un puerto para recibir los mensajes por UDP
    puerto_udp = randint(10000, 30000)
    # En caso de que se elija una opción válida, debes crear el socket UDP
    # Pista: revisa la última parte del notebook 2 para ver cómo se crea un
    # socket UDP y se configura para recibir mensajes
    # COMPLETAR
    socket_udp = 
    
    

    # También, debes enviar el comando "hora", junto con la "ciudad" y el "puerto"
    # Recuerda que debes respetar el formato de los mensajes pedido en el enunciado
    # COMPLETAR
    
    
    

    try:
        print("Mostrando hora (presiona Ctrl + C para detener)")
        while True:
            # El siguiente código es el encargado de recibir la hora y mostrarla
            mensaje_recibido, _ = socket_udp.recvfrom(1024)
            dict_recibido = json.loads(mensaje_recibido)
            hora = dict_recibido["hora"]
            print(f"La hora en {opciones[opcion]} es: {hora}", end="\r")
    except ConnectionError:
        print("Hubo un error de conexión!")
    except KeyboardInterrupt:
        print("\nDejando de recibir hora...")
    finally:
        # En caso de que se presione Ctrl + C, el servidor deberá dejar de enviar
        # la hora, y el cliente deberá dejar de recibirla
        # Por lo tanto, a continuación debes cerrar el socket UDP, y
        # enviar el comando "detener"
        # COMPLETAR
        
        
        
        
        


El siguiente corresponde al código del **servidor**. Este se implementa mediante una clase `Server`, la cual viene con la función `obtener_string_hora` ya implementada. Esta recibe una ciudad, y retorna un string que contiene la hora de dicha ciudad, en el formato `"hh:mm:ss.msmsms"`.

Los mensajes que el servidor **envía** al cliente por medio de UDP también están codificados en JSON, y deben tener el siguiente formato:
```python
{
    "hora" : string_hora
}
```

A continuación, deberás completar las partes que están sin implementar del código:

In [ ]:
# Release
from datetime import datetime, timezone, timedelta
import socket
import json
import threading


class Server:

    zonas_horarias = {
        "Santiago" : -4,
        "Sao Paulo" : -3,
        "Ciudad de México" : -5,
        "New York" : -4,
        "Los Angeles" : -7,
        "Madrid" : +2,
        "Londres" : +1,
        "Berlin" : +2,
        "Moscú" : +3,
        "Ciudad del Cabo" : +2,
        "Delhi" : +5.5,
        "Beijing" : +8,
        "Singapur" : +8,
        "Sydney" : +10
    }

    def __init__(self, host, port):
        self.host_tcp = host
        self.puerto_tcp = port
        self.seguir_enviando = False
        # A continuación debes crear el socket UDP
        # Pista: revisa la sección Cliente UDP del notebook 2 para ver un ejemplo
        # de cómo se crea un socket UDP que pueda enviar
        # COMPLETAR
        self.socket_udp = 
        
        # Luego, debes crear el socket TCP, y conectarlo para que reciba clientes
        # COMPLETAR
        self.socket_tcp = 
        
        
        
        # Aquí tu socket deberá ser capaz de recibir un cliente
        # COMPLETAR
        self.socket_cliente, address_cliente = 
        
        self.host_cliente = address_cliente[0]
        print(f"Se ha recibido el cliente {self.host_cliente}")
        self.escuchar_cliente()

    def obtener_string_hora(self, ciudad):
        """
        Recibe una ciudad como parámetro.
        Retorna un string que contiene la hora de dicha ciudad, con el formato:
        "hh:mm:ss.msmsms"
        """
        zona_horaria = self.zonas_horarias[ciudad]
        hora = datetime.now(timezone(timedelta(hours=zona_horaria))).time()
        return str(hora)

    def escuchar_cliente(self):
        print("Escuchando al cliente...")
        while True:
            # Aquí debes recibir el mensaje del cliente (el socket debería estar
            # en el atributo self.socket_cliente). También debes decodificar el
            # mensaje JSON que te envía
            # COMPLETAR
            mensaje_recibido = 
            
            dict_recibido = 
            
            print("Cliente envió:", dict_recibido)
            if dict_recibido["comando"] == "hora":
                self.seguir_enviando = True
                # Se crea un thread que envía continuamente la hora al cliente
                # por medio de UDP
                thread_udp = threading.Thread(
                    target=self.enviar_hora,
                    args=(dict_recibido, )
                    )
                thread_udp.start()
            elif dict_recibido["comando"] == "detener":
                # Si se recibe el comando detener, se detiene el thread
                self.seguir_enviando = False
            elif dict_recibido["comando"] == "cerrar":
                # En caso de cerrar el servidor, no olvides cerrar ámbos sockets
                # COMPLETAR
                
                
                break

    def enviar_hora(self, dict_):
        ciudad = dict_["ciudad"]
        puerto = dict_["puerto"]
        while self.seguir_enviando:
            # Aquí debes enviar la hora al cliente por medio de UDP
            # El host del cliente está en self.host_cliente
            # Recuerda que tienes el método obtener_string_hora, y que debes
            # seguir el formato pedido en el enunciado
            # COMPLETAR
            
            
            
            
            


if __name__ == "__main__":
    HOST = socket.gethostname()
    PORT = 35020

    SERVER = Server(HOST, PORT)


## Ejercicio 3: Haciendo "eco" a un cliente

El malvado emperador EnZurg, hermano gemelo malvado de Enzo, ha desterrado a Enzo a las lejanas tierras del DCCampo. Hace un tiempo, por razones desconocidas, EnZurg está intentando contactar de nuevo Enzo a través de su servidor privado, pero este no quiere responderle y prefiere hacerlo perder el tiempo por haberlo desterrado. En este ejercicio deberás completar el servidor de Enzo.

El servidor de Enzo funciona de una forma bastante especial. Cada vez que el cliente de EnZurg que se conecta le envía un mensaje, Enzo responde el mismo contenido (un "eco") pero con mayúsculas y minúsculas puestas al azar, deformando el *string*. Algo así:

> EnZurg: Hola, Enzo
> 
> Enzo: HoLa, eNzO

Para esto, Enzo te entrega su función `deformador_string`. Finalmente, como Enzo creció junto con EnZurg, conoce a la perfección el código del cliente que está utilizando su hermano (podría serte útil para probar tu servidor). **Es tiempo de completar el servidor para molestar a EnZurg.**

In [ ]:
# Código funcion deformador_string
import random


def deformador_string(string):
    string_deformado = ""
    for caracter in string:
        if random.random() <= 0.5:
            string_deformado += caracter.upper()
        else:
            string_deformado += caracter.lower()
    return string_deformado

print(deformador_string("Hola, Enzo"))


In [ ]:
# Código cliente de EnZurg
import socket


# Creo un socket TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así puedo obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9001

# Me conecto al socket del servidor de Enzo
sock.connect((host, port))

try:
    while True:
        # Escribo el mensaje a enviarle a Enzo
        mensaje = input("Yo a Enzo: ")
        # Si escribo "salir", se corta la conexión
        if mensaje.lower() == "salir":
            sock.sendall("salir".encode("utf-8"))
            break
        # Envio mi mensaje al servidor
        sock.sendall(mensaje.encode("utf-8"))
        # Recibo la respuesta que me envíe
        data = sock.recv(4096)
        print("Enzo a mi:", data.decode("utf-8"))
except ConnectionError as e:
    print("Ocurrió un error.")
finally:
    sock.close()


In [ ]:
# Código servidor de Enzo
import random
import socket

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9001

# Dejamos el socket esperando ("escuchando") por conexiones
sock.bind((host, port))
sock.listen()

# Aqui aceptamos la conexión de EnZurg
socket_cliente, address = sock.accept()


def deformador_string(string):
    string_deformado = ""
    for caracter in string:
        if random.random() <= 0.5:
            string_deformado += caracter.upper()
        else:
            string_deformado += caracter.lower()
    return string_deformado


# COMPLETAR AQUI
# Debes recibir el mensaje de EnZurg,
# aplicarle la función deformador_string
# y enviar ese string modificado de vuelta a EnZurg


## Ejercicio 4: Envío de mensajes desde el servidor

En un nuevo intento por saber de Enzo, EnZurg decide dejar de intentar contactarlo y simplemente esperar a que le envíe él un mensaje. Es por esto que se te ocurre ayudar a Enzo enviando *spam* al cliente de EnZurg que se intentará conectar al servidor de Enzo, simplemente para llenarlo de basura e información no muy útil.

En este ejercicio, deberás volver a implementar un servidor, pero deberá enviar un mensaje con *spam* **cada 5 segundos** al cliente que se conecte.

In [ ]:
# Código cliente de EnZurg
import socket

# Creo un socket TCP
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así puedo obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9002

# Me conecto al socket del servidor de Enzo
sock.connect((host, port))

try:
    while True:
        # Voy a ir recibiendo sus mensajes
        data = sock.recv(4096)
        # Y ahora los voy a ir imprimiendo
        print("Enzo a mi:", data.decode("utf-8"))
except ConnectionError as e:
    print("Ocurrió un error.")
finally:
    sock.close()


In [ ]:
# Código servidor de Enzo
import random
import socket
import time

sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

# Así podemos obtener el hostname de la máquina actual
host = socket.gethostname()
port = 9002

# Dejamos el socket esperando ("escuchando") por conexiones
sock.bind((host, port))
sock.listen()

# Aqui aceptamos la conexión de EnZurg
socket_cliente, address = sock.accept()


MENSAJES_SPAM = [
    "SPAM!! SPAM!! SPAM!! SPAM!! SPAM!!",
    "lalalalalalalalalalalalala",
    "Intentando sobrecargar el cliente de EnZurg",
    "La respuesta a la Vida, el Universo, y Todo lo Demás es... 42",
]

# COMPLETAR AQUI
# Cada 5 segundos debes enviar un mensaje
# de la lista MENSAJES_SPAM a EnZurg


## Ejercicio 5: Definiendo un protocolo más formal de comunicación

Finalmente, Enzo decide aceptar conversar con EnZorg. Como su relación es bastante complicada, deciden establecer un protocolo de comunicación formal para no tener problemas de comunicación en el futuro. Lamentablemente, como ambos son bastante tercos, este sistema que podría ser bastante simple termina siendo más complejo de lo necesario (algunos dicen que por su complejidad podría ser utilizado en una tarea).

Las reglas del protocolo son las siguientes:
* Los primeros 4 bytes de un mensaje indicarán el largo del contenido del mensaje, los que tendrán que estar en formato *little endian*
* El contenido del mensaje debe ser dividido en tantos bloques de 80 *bytes* como sea necesario, y para cada bloque se deberán anteponer 4 *bytes* que indiquen el número de bloque, codificado en *big endian*.
* Para transformar *strings* a *bytes* se deberá utilizar UTF-8.

En este ejercicio, deberás primero hacer funciones que permitan trabajar fácilmente con el protocolo (parte A) y luego implementar un cliente y un servidor que utilice dicho protocolo (parte B), que serán utilizados por Enzo y EnZurg desde ahora en adelante.

### Parte A: Implementación del protocolo

Primero que todo, el [*endianness*](https://es.wikipedia.org/wiki/Endianness) se refiere al orden en que se guardan los *bytes* que representan un número. Dado que un número puede requerir representarse con más de un *byte*, es importante saber si primero tenemos los bytes más significativos o los menos significativos. 

Cuando convertimos un número a *byte*, podemos elegir cuántos *byte* utilizar (primer argumento) y el *endianness* (`byteorder`) de su representación (segundo argumento):

In [2]:
(1025).to_bytes(4, byteorder="little") # little endian

b'\x01\x04\x00\x00'

In [3]:
(1025).to_bytes(4, byteorder="big") # big endian

b'\x00\x00\x04\x01'

Como puedes ver, estamos representando el número 1025 con 4 *bytes* en ambas celdas, pero el orden de los *bytes* en cada celda cambia.

A continuación, se hará un ejemplo paso a paso del protocolo.

In [ ]:
# Este es el mensaje de ejemplo (sí, es una receta para hacer sopaipillas)
MENSAJE = ("Lo primero que hay que tener en cuenta para saber cómo hacer "
           "sopaipillas, es hacer un “volcán” con la harina dejando un hueco "
           "al medio, vierta al centro la manteca derretida con la leche o el "
           "agua, la sal y el zapallo previamente molido hasta formar una "
           "pasta suave, mezcle todo hasta formar una masa suave y elástica, "
           "esta no debe tener grumos y no se tiene que pegar a la mesa.\n"
           "\n"
           "El siguiente paso para saber cómo hacer sopaipillas es amasar la "
           "masa con un uslero o a mano dejándola de aproximadamente 5mm de "
           "grosor, luego  cortarla en  círculos de 10cm (esto es a gusto, "
           "pueden ser más gruesas y más pequeñas), se recomienda pincharlas "
           "con un tenedor para que al momento de freírlas la masa no se "
           "arruine.\n"
           "\n"
           "El último paso es freír la masa en un sartén profundo y caliente "
           "el aceite a fuego alto, para probar el aceite lance un pequeño "
           "trozo de masa, debe burbujear y flotar en la superficie, coloque "
           "de 2 a 3 sopaipillas y fríalas 1 minuto por lado, no se deben "
           "dorar demasiado. Una vez fritas se sacan de la freidora y dejan en "
           "papel absorbente.\n"
           "\n"
           "Luego de haber leído esta receta de comida típica chilena, usted "
           "ya habrá sabido cómo hacer sopaipillas.")

**Paso 0:** Convertir el contenido a *bytes* (usando UTF-8)

**Paso 1:** Obtener el largo del contenido y representar dicho largo en 4 *bytes* (*little endian*). Este es el **primer bloque**

**Paso 2:** Dividir el contenido en bloques de 80 *bytes*, y mantener el orden de los bloques
* **Paso 2a:** En este mismo paso, se puede convertir el índice del bloque a su representación en el protocolo (4 *bytes* en formato *big endian*)
* **Paso 2b:** Conservar cada bloque (ya sea de contenido o su posición) en su posición correcta (a partir del primer bloque)

> *Hint*: Recuerda la sección _chunks_ del notebook de serialización

Ahora, deberás crear **una función `codificar`** que retorne la lista de bloques a enviar, con la codificación del contenido ya explicada.

In [ ]:
def codificar(contenido):
    contenido_como_bytes = None # Paso 0 (COMPLETAR)
    largo = len(contenido_como_bytes)
    largo_como_bytes = None # Paso 1 (COMPLETAR)
    BLOQUES = [largo_como_bytes]
    for ... in ...: # Paso 2 (COMPLETAR)
        posicion_como_bytes = None # Paso 2a (COMPLETAR)
        chunk_como_bytes = None # Paso 2 (COMPLETAR)
        BLOQUES.append(posicion_como_bytes) # Paso 2b
        BLOQUES.append(chunk_como_bytes) # Paso 2b
    return BLOQUES

In [ ]:
codificar(MENSAJE)

### Parte B: Usando el protocolo en el servidor y el cliente

Ahora, basandote en el código de cliente y servidor utilizado en los pasos anteriores, deberás implementar un servidor y un cliente que utilicen el protocolo. Presta especial atención al uso de los datos en cada parte. Como notarás, deberás recibir primero los bytes que indiquen el largo del mensaje, estimar la cantidad de bloques y para cada bloque recibir su respeciva posición y contenido.